In [ ]:

import cv2
import sys
import glob
import pandas as pd
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import f1_score
import os
from PIL import Image
import xlrd 
import math
import pylab
#from scipy import signal
import numpy as np
from matplotlib import pyplot as plt
#from skimage.feature import greycomatrix, greycoprops
#from skimage import data
import math
#from imutils import contours
#from skimage import measure
import argparse
import imutils

In [ ]:
def resize(image,width,height):
    dim = (width,height)
    res = cv2.resize(image, dim, interpolation=cv2.INTER_LINEAR)
    return res

def fix(i1,i2,max):
    if(i1 < 10):
        return 0, i2-i1
    if(i2>max):
        return i1-i2+max, max
    return i1,i2

def crop(image):
    width,height,n = image.shape
    loc = cv2.minMaxLoc(image[:,:,1])
    x = loc[3][0]
    y = loc[3][1]
    size = width/3

    y1, y2, x1, x2 = y-int(size/2), y+int(size/2), x-int(size/2), x+int(size/2)


    y1,y2 = fix(y1,y2,width)
    x1,x2 = fix(x1,x2,height)

    



    cropPic = image[y1:y2,x1:x2,]

    return cropPic


def read(filename):
    img = cv2.imread(filename)
    return img[:,:,::-1]

def cvtGray(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return gray

def showRGB(image):
    r = image[:,:,0]
    g = image[:,:,1]
    b = image[:,:,2]
    
    plt.subplot(2,2,1)
    plt.imshow(image)
    plt.subplot(2,2,2)
    plt.imshow(r,cmap='gray')
    plt.subplot(2,2,3)
    plt.imshow(g,cmap='gray')
    plt.subplot(2,2,4)
    plt.imshow(b,cmap='gray')
    plt.show()
    
def greenRatio(image):
    green = image[:,:,1]
    return np.sum(green)/np.sum(image)

import csv
import random
def writeCsv(fname, data, *args, **kwargs):
    mycsv = csv.writer(open(fname, 'w'), *args, **kwargs)
    mycsv.writerow(["area","diameter","label"])
    for row in data:
        mycsv.writerow(row)
    
def showRGB(image):
    r = image[:,:,0]
    g = image[:,:,1]
    b = image[:,:,2]
    
    plt.subplot(1,4,1)
    plt.imshow(image)
    plt.subplot(1,4,2)
    plt.imshow(r,cmap='gray')
    plt.subplot(1,4,3)
    plt.imshow(g,cmap='gray')
    plt.subplot(1,4,4)
    plt.imshow(b,cmap='gray')
    plt.show()

In [ ]:
def vessel_remove(image):
    qq = 4
    image2 = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.medianBlur(gray, 5)
    adapt_type = cv2.ADAPTIVE_THRESH_GAUSSIAN_C
    thresh_type = cv2.THRESH_BINARY_INV
    bin_img = cv2.adaptiveThreshold(blur, 255, adapt_type, thresh_type, 31, 5)
    bini = cv2.cvtColor(bin_img, cv2.COLOR_BGR2RGB)
    gg = cv2.cvtColor(bini,cv2.COLOR_BGR2GRAY)
    
    rr,cc = image2.shape[:2]
    
    r,g,b = cv2.split(image2)
    
    for i in range(1,rr):
            for j in range(1,cc):
                if gg[i,j] == 255:
                    i1,i2 = fix(i-qq,i+qq,rr)
                    j1,j2 = fix(j-qq,j+qq,cc)
                    image2[:,:,0][i,j] = np.amax(r[i1:i2,j1:j2])
                    image2[:,:,1][i,j] = np.amax(g[i1:i2,j1:j2])
                    image2[:,:,2][i,j] = np.amax(b[i1:i2,j1:j2])
    for i in range(1,rr):
            for j in range(1,cc):
                    i1,i2 = fix(i-4,i+4,rr)
                    j1,j2 = fix(j-4,j+4,cc)
                    image2[:,:,0][i,j] = np.amax(r[i1:i2,j1:j2])
                    image2[:,:,1][i,j] = np.amax(g[i1:i2,j1:j2])
                    image2[:,:,2][i,j] = np.amax(b[i1:i2,j1:j2])
    
    return image2
        
    

In [ ]:
def vessel_count(image):
    image2 = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.medianBlur(gray, 5)
    adapt_type = cv2.ADAPTIVE_THRESH_GAUSSIAN_C
    thresh_type = cv2.THRESH_BINARY_INV
    bin_img = cv2.adaptiveThreshold(blur, 255, adapt_type, thresh_type, 31, 5)
    bini = cv2.cvtColor(bin_img, cv2.COLOR_BGR2RGB)
    gg = cv2.cvtColor(bini,cv2.COLOR_BGR2GRAY)
    return np.count_nonzero(gg == 255)

In [ ]:
def segment(image):
    Aro,Ago,Abo = cv2.split(image)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))
    Aro = clahe.apply(Aro)
    Ago = clahe.apply(Ago)
    M = 60
    sd = Ago.std()
    filter = signal.gaussian(M, std=6)
    filter=filter/sum(filter)
    STDf = filter.std()
    
    Ar = Aro - Aro.mean() - Aro.std() 
    
    Mr = Ar.mean()                           
    SDr = Ar.std()                          
    Thr = 0.5*M - STDf - Ar.std()        
    
    hist,bins = np.histogram(Ag.ravel(),256,[0,256])   
    
    smooth_hist_g=np.convolve(filter,hist)  
    smooth_hist_r=np.convolve(filter,histr)
    
    r,c = Ag.shape
    Dd = np.zeros(shape=(r,c))
    Dc = np.zeros(shape=(r,c))
    
    Thg = np.amax(Ago)-sd*1.3+50-sd*1.61
    
    for i in range(1,r):
        for j in range(1,c):
            if Ar[i,j]>Thr:
                Dd[i,j]=255
            else:
                Dd[i,j]=0

    for i in range(1,r):
        for j in range(1,c):
        
            if Ago[i,j]>Thg:
                Dc[i,j]=1
            else:
                Dc[i,j]=0
                
    kernel = np.ones((2,2),dtype=np.float32)
    Dd = resize(Dd,20,20)
    Dd = resize(Dd,r,c)

    Dc = cv2.dilate(Dc,kernel,iterations = 2)
    Dc = cv2.erode(Dc,kernel,iterations = 3)
    Dc = resize(Dc,r,c)
            
    cv2.imwrite('disk.png',Dd)
    plt.imsave('cup.png',Dc)
    
        
import cv2 as cv

def cdr(image,label):
    try:
        cup = read("cup.png")
        disc = read("disk.png")
        image1 = image.copy()
        image2 = image.copy()
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))

        R1 = cv2.morphologyEx(cup, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
        r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
        R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
        r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
        R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)	
        r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)
        r3 = cv.cvtColor(r3, cv.COLOR_BGR2GRAY)

        img = clahe.apply(r3)

        cup = cv.cvtColor(cup, cv.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(cup,127,255,0)
        contours,hierarchy = cv2.findContours(thresh, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)


        cup_diameter = 0
        largest_area = 0
        el_cup = contours[0]
        if len(contours) != 0:
            for i in range(len(contours)):
                if len(contours[i]) >= 5:
                    area = cv.contourArea(contours[i])
                    if (area>largest_area):
                        largest_area=area
                        index = i
                        el_cup = cv.fitEllipse(contours[i])

        cv.ellipse(image1,el_cup,(140,60,150),3)
        x,y,w,h = cv2.boundingRect(contours[index])
        cup_diameter = max(w,h)
        ac,bc = max(w,h),min(w,h)

        R1 = cv2.morphologyEx(disc, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations = 1)
        r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)), iterations = 1)
        R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(1,21)), iterations = 1)
        r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(21,1)), iterations = 1)
        R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(33,33)), iterations = 1)
        r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(43,43)), iterations = 1)
        #f4 = cv2.subtract(R3,img)
        r3 = cv.cvtColor(r3, cv.COLOR_BGR2GRAY)
        img2 = clahe.apply(r3)
        disc = cv.cvtColor(disc, cv.COLOR_BGR2GRAY)
        ret,thresh = cv.threshold(disc,127,255,0)
        contours,hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)

        disk_diameter = 0
        largest_area = 0
        el_disc = el_cup
        if len(contours) != 0:
              for i in range(len(contours)):
                if len(contours[i]) >= 5:
                    area = cv.contourArea(contours[i])
                    if (area>largest_area):
                        largest_area=area
                        index = i
                        el_disc = cv.fitEllipse(contours[i])
        image2 = image1.copy()               
        cv.ellipse(image2,el_disc,(140,60,150),3)
        x,y,w,h = cv2.boundingRect(contours[index])
        ad,bd = max(w,h), min(w,h)
        disk_diameter = max(w,h)

        if False:
            plt.subplot(1,2,2)
            plt.imshow(image2, cmap = 'gray', interpolation = 'bicubic')
            plt.axis("off")
            plt.title("Disk "+label)
            plt.subplot(1,2,1)
            plt.imshow(image1)
            plt.axis("off")
            plt.title("Cup "+label)
            plt.show()

        if(disk_diameter == 0): return 1
        cdr = (math.pi*ac*bc)/(math.pi*ad*bd)
        return cdr,bc/ac
    except:
        print("error")
        return -1,-1

In [ ]:
import pickle
import time as t
start = t.time()
dataset = []
path = r"D:\dataset\dataProceed2\data"
count = 0
for label in os.listdir(path):
    for filename in os.listdir(os.path.join(path,label)):
        feature = []
        loc = os.path.join(path,label,filename)
        image = resize(read(loc),240,240)
        image2 =vessel_remove(image)
        segment(image2)
        cdrr,cdrd = cdr(image2,label)
        if cdrr == -1 or cdrr>1.3: continue
        feature.append(findhog(image))
        feature.append(cdrr)
        dataset.append([feature,label])
        count += 1
        if count%10 == 0 or count == 1: 
            print(count,"pictures done...")
        
with open("dataset3.pkl",'wb') as file:
    pickle.dump(dataset,file)
last = t.time()

print("finished with time used : "+str((last-start)/60)+" minutes.")

In [ ]:
with open("dataset3.pkl",'rb') as file:
    j = pickle.load(file)
datasett = []
for i in j:
    kk = []
    kk.append(i[0][0])
    kk.append(i[0][1])
    kk.append(i[1])
    datasett.append(np.asarray(kk))
with open("datapickle.pkl",'wb') as file:
    pickle.dump(np.asarray(datasett),file)

In [ ]:
datasett[0][2]

In [ ]:
import random as r
with open("datapickle.pkl",'rb') as file:
    test1 = list(pickle.load(file))
r.shuffle(test1)
g = []
n = []
o = []
all = {'G':g,"N":n,"O":o}
lab = ["G","N","O"]
for data in test1:
    labe = data[2]
    if labe == "G":
        k = []
        k.append(data[0])
        k.append(1)
        g.append(k)
    else:
        k = []
        k.append(data[0])
        k.append(0)
        g.append(k)
    if labe == "N":
        k = []
        k.append(data[0])
        k.append(1)
        n.append(k)
    else:
        k = []
        k.append(data[0])
        k.append(0)
        n.append(k)
    if labe == "O":
        k = []
        k.append(data[0])
        k.append(1)
        o.append(k)
    else:
        k = []
        k.append(data[0])
        k.append(0)
        o.append(k)
    
for fname in ["G","N","O"]:
    with open(fname+".pkl",'wb') as file:
        pickle.dump(np.asarray(all[fname]),file)
    

In [ ]:
import csv
import random
def writeCsv23(fname, data, *args, **kwargs):
    mycsv = csv.writer(open(fname, 'w'), *args, **kwargs)
    mycsv.writerow(["area","label"])
    for row in data:
        mycsv.writerow(row)


In [ ]:
import random as r
with open("dataset2.pkl",'rb') as file:
    test1 = pickle.load(file)
    print(len(test1))
print(np.asarray(test1[:10]))

In [ ]:
print(np.asarray(test1[500:550]))
a = [print(x[0[0]])]

In [ ]:
from sklearn.svm import SVC
import sklearn
def readCSV(fname):
    features = []
    label = []
    table = pd.read_csv(fname)
    for i in range(len(table.index)):
        features.append(list(table.iloc[i])[:-1])
        label.append(int(list(table.iloc[i])[-1]))
    features_array = np.asarray(features)
    label_array = np.asarray(label)
    #,"diameter","green_ratio","blood vessel"
    fname = ["area"
             ,"diameter","label"]
    return sklearn.utils.Bunch(features = features_array, label = label_array,features_name = fname)

In [ ]:
data = readCSV("G.csv")

In [ ]:
print(data.label)

In [ ]:
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt

def findhog(image):
    fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, multichannel=True)
    return fd

In [ ]:
print(fd.shape)

In [12]:
import cv2
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt

def findhog(image):
    fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), visualize=True, multichannel=True)
    return fd

def predict(image):
    x,y = image[:,:,1].shape
    if np.count_nonzero(image < 4)/(x*y) > 0.2:
        image = crop(image)
    image = resize(image,240,240)
    label = ['glaucoma','normal','others']
    feature = np.asarray(findhog(image))

    with open("svmModel.pkl",'rb') as file:
        modelList = pickle.load(file)

    labelG = modelList[0].predict(feature)
    labelN = modelList[1].predict(feature)
    labelO = modelList[2].predict(feature)

    label = {"glaucoma":labelG,"normal":labelN,"Others":labelO}

    prediction = ""

    if labelG+labelN+lebelO == 1:
        for i in label:
            if label[i] == 1: 
                prediction = i
    elif labelG+labelN+lebelO == 2:
        l = []
        for i in label:
            if label[i] == 1: 
                l.append(i)
        if 'normal' in l: prediction = i
        else: prediction = l[0]
    elif labelG+labelN+lebelO == 3:
        prediction = "others"

    else: prediction = 'normal'

    return prediction

image = cv2.imread("38_left.jpg")[:,:,::-1]
image = crop(image)
print(predict(image))


AttributeError: module 'scipy' has no attribute '_lib'

In [ ]:
def claheAdapter(img):
    lab_img= cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab_img)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(l)
    updated_lab_img2 = cv2.merge((clahe_img,a,b))
    CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2RGB)
    return CLAHE_img

plt.imshow(claheAdapter(cv2.imread("38_left.jpg")[:,:,::-1]))

In [ ]:
import os
clah = []
path = r"D:\dataset\dataProceed2\data\G"
for filename in os.listdir(path)[135:150]:
    image = cv2.imread(os.path.join(path,filename))[:,:,::-1]
    cl = claheAdapter(image)
    segment(image)


In [ ]:
with open("G.pkl",'rb') as file:
    test1 = pickle.load(file)
for i in test1: print(i[1])